#### 各种库

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns

#### 路径简化

In [2]:
base_path = "D:/CODE/datecontest/Study.test/2024年钉钉杯大数据挑战赛初赛题目/A题/"
file_names = ["A1.csv", "A2.csv", "A3.csv", "A4.csv", "A5.csv"]

In [3]:
dfs = [pd.read_csv(base_path + fname, encoding='utf-8-sig') for fname in file_names]
names = ['A1', 'A2', 'A3', 'A4', 'A5']

#### 类型转换

- 每张表的列数据类型

In [4]:
for name, df in zip(names, dfs):
    print("表格类型：\n", df.dtypes)
    print("基本信息：\n", df.info())
    print("=========================================")

表格类型：
 月份         int64
样品代码     float64
名称        object
销量（箱）    float64
金额（元）    float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   月份      137 non-null    int64  
 1   样品代码    117 non-null    float64
 2   名称      117 non-null    object 
 3   销量（箱）   117 non-null    float64
 4   金额（元）   117 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 5.5+ KB
基本信息：
 None
表格类型：
 月份         int64
样品代码     float64
名称        object
销量（箱）    float64
金额（元）    float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   月份      108 non-null    int64  
 1   样品代码    98 non-null     float64
 2   名称      98 non-null     object 
 3   销量（箱）   98 non-null     float64
 4   金额（元）   98 non-null 

- 检测异常类型并转换

In [5]:
def convert_types(df):
    # 转月份为日期
    df['日期'] = pd.to_datetime(df['月份'].astype(str), format='%Y%m', errors='coerce')

    # 样品代码转字符串
    df['样品代码'] = df['样品代码'].astype('Int64').astype(str)

    # 数值转换
    df['销量（箱）'] = pd.to_numeric(df['销量（箱）'], errors='coerce')
    df['金额（元）'] = pd.to_numeric(df['金额（元）'], errors='coerce')

    return df

for df in dfs:
    df = convert_types(df)

  - 补全月份

In [6]:
def preprocess_and_check_missing_months(df, date_col='月份', date_format='%Y%m'):
    # 1. 转换为标准日期列（每月第一天）
    df['日期'] = pd.to_datetime(df[date_col].astype(str), format=date_format, errors='coerce')

    # 2. 构造完整月份索引（按月）
    full_months = pd.date_range(start=df['日期'].min(), end=df['日期'].max(), freq='MS')

    # 3. 获取原始数据中已存在的月份
    present_months = pd.to_datetime(df['日期'].dropna().dt.to_period('M').astype(str), format='%Y-%m')

    # 4. 找出缺失的月份（原数据中未出现）
    missing_months = sorted(set(full_months) - set(present_months))

    # 5. 返回结构
    return {
        "起止范围": (full_months.min(), full_months.max()),
        "总应有月份数": len(full_months),
        "实际月份数": len(present_months.unique()),
        "缺失月份数": len(missing_months),
        "缺失月份列表": [dt.strftime('%Y-%m') for dt in missing_months]
    }

# 批量处理
for name, df in zip(names, dfs):
    print(f"===== 处理 {name} 数据集 =====")
    result = preprocess_and_check_missing_months(df)
    print(f"起止范围：{result['起止范围'][0].strftime('%Y-%m')} 到 {result['起止范围'][1].strftime('%Y-%m')}")
    print(f"应有月份数：{result['总应有月份数']}，实际有数据的月份数：{result['实际月份数']}")
    print(f"缺失月份数：{result['缺失月份数']}，缺失月份如下：")
    print(result['缺失月份列表'])
    print("="*40)


===== 处理 A1 数据集 =====
起止范围：2011-01 到 2022-07
应有月份数：139，实际有数据的月份数：137
缺失月份数：2，缺失月份如下：
['2021-01', '2022-01']
===== 处理 A2 数据集 =====
起止范围：2011-01 到 2020-05
应有月份数：113，实际有数据的月份数：108
缺失月份数：5，缺失月份如下：
['2013-11', '2014-12', '2015-04', '2017-12', '2020-01']
===== 处理 A3 数据集 =====
起止范围：2014-06 到 2023-11
应有月份数：114，实际有数据的月份数：109
缺失月份数：5，缺失月份如下：
['2015-04', '2020-12', '2021-12', '2022-12', '2023-01']
===== 处理 A4 数据集 =====
起止范围：2011-01 到 2024-01
应有月份数：157，实际有数据的月份数：135
缺失月份数：22，缺失月份如下：
['2018-02', '2018-12', '2019-02', '2019-12', '2020-11', '2020-12', '2021-01', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02', '2022-08', '2022-10', '2022-11', '2022-12', '2023-05', '2023-07', '2023-08', '2023-09', '2023-10', '2023-11']
===== 处理 A5 数据集 =====
起止范围：2011-01 到 2024-04
应有月份数：160，实际有数据的月份数：105
缺失月份数：55，缺失月份如下：
['2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01', '2020-0

In [7]:
import pandas as pd

def preprocess_for_forecast(df, date_col='月份'):
    # 1. 转换“月份”列为 datetime
    df = df.copy()
    df['日期'] = pd.to_datetime(df[date_col].astype(str), format='%Y%m')
    
    # 2. 找到最后一个有销量的观测点
    last_obs = df.loc[df['销量（箱）'].notna(), '日期'].max()
    
    # 3. 拆分训练集 / 预测集
    train = df[df['日期'] <= last_obs].copy()
    pred  = df[df['日期'] >  last_obs].copy()
    
    # 4. 补齐训练集中的所有断月
    full_idx = pd.date_range(start=train['日期'].min(),
                             end=last_obs,
                             freq='MS')
    train = train.set_index('日期').reindex(full_idx).rename_axis('日期').reset_index()
    
    # 5. 填充训练集缺失值
    # 类别列前向填充
    train['样品代码'] = train['样品代码'].ffill()
    train['名称']    = train['名称'].ffill()
    # 数值列线性插值
    train['销量（箱）'] = train['销量（箱）'].interpolate()
    train['金额（元）'] = train['金额（元）'].interpolate()
    
    # 6. 还原“月份”列（如果需要）
    train['月份'] = train['日期'].dt.strftime('%Y%m').astype(int)
    pred ['月份'] = pred ['日期'].dt.strftime('%Y%m').astype(int)
    
    return train, pred

# ===== 用法示例 =====
df = pd.read_csv(base_path + file_names[1], encoding='utf-8-sig')
train_df, pred_df = preprocess_for_forecast(df)

print("训练集：", train_df.shape, "最早→最晚：", train_df['日期'].min(), train_df['日期'].max())
print("预测集：", pred_df.shape, "含空白月份：", pred_df['日期'].tolist())


训练集： (102, 6) 最早→最晚： 2011-01-01 00:00:00 2019-06-01 00:00:00
预测集： (10, 6) 含空白月份： [Timestamp('2019-07-01 00:00:00'), Timestamp('2019-08-01 00:00:00'), Timestamp('2019-09-01 00:00:00'), Timestamp('2019-10-01 00:00:00'), Timestamp('2019-11-01 00:00:00'), Timestamp('2019-12-01 00:00:00'), Timestamp('2020-02-01 00:00:00'), Timestamp('2020-03-01 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2020-05-01 00:00:00')]
